# ¿Cuál es la mejor tarifa?

Trabajas como analista para el operador de telecomunicaciones Megaline. La empresa ofrece a sus clientes dos tarifas de prepago, Surf y Ultimate. El departamento comercial quiere saber cuál de las tarifas genera más ingresos para poder ajustar el presupuesto de publicidad.

Vas a realizar un análisis preliminar de las tarifas basado en una selección de clientes relativamente pequeña. Tendrás los datos de 500 clientes de Megaline: quiénes son los clientes, de dónde son, qué tarifa usan, así como la cantidad de llamadas que hicieron y los mensajes de texto que enviaron en 2018. Tu trabajo es analizar el comportamiento de los clientes y determinar qué tarifa de prepago genera más ingresos.

## Inicialización

In [1]:
# Cargar todas las librerías
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats as st
import seaborn as sns



## Cargar datos

In [2]:
# Carga los archivos de datos en diferentes DataFrames
megaline_calls = pd.read_csv("/datasets/megaline_calls.csv")
megaline_internet = pd.read_csv("/datasets/megaline_internet.csv")
megaline_messages = pd.read_csv("/datasets/megaline_messages.csv")
megaline_plans = pd.read_csv("/datasets/megaline_plans.csv")
megaline_users = pd.read_csv("/datasets/megaline_users.csv")




FileNotFoundError: [Errno 2] No such file or directory: '/datasets/megaline_calls.csv'

## Preparar los datos

In [ ]:
#En este espacio vamos a verificar si el dataset de tarifas tiene valores duplicados o ausentes
print(megaline_plans.duplicated().sum())
print(megaline_plans.isna().sum())

In [ ]:
#verificar si el dataset de tarifas tiene valores duplicados o ausentes
print(megaline_users.duplicated().sum())
print(megaline_users.isna().sum())
print(megaline_users.nunique())
print(megaline_users["city"].unique())

## Tarifas

In [ ]:
# Imprime la información general/resumida sobre el DataFrame de las tarifas
megaline_plans.info()


In [ ]:
# Imprime una muestra de los datos para las tarifas
print(megaline_plans.head())





Hasta el momento no veo datos no adecuados, ausentes o duplicados.

## Usuarios/as

In [ ]:
# Imprime la información general/resumida sobre el DataFrame de usuarios
megaline_users.info()


In [ ]:
# Imprime una muestra de datos para usuarios
megaline_users.sample(40)



En el primer momento que vi esta pregunta no tenia mucho que agragar ya que no sabia que tenia que hacer con los datos, a lo largo del proyecto necesite crear columnas nuevas con nuevas informaciones para poder hacer los analisis respectivos.

### Corregir los datos

In [ ]:
#Vamos a verificar que todas se escriban de la misma manera
megaline_users["city"] = megaline_users["city"].str.title()
#luego verificamos si cambiaron los valores unicos
print(megaline_users.nunique())
megaline_users["city"]

### Enriquecer los datos

En este dataset hemos encontrado una cantidad importante de valores ausentes, en la misma columna vemos que tenemos fechas que no estan en tipo datetime, sin embargo hasta este punto no voy a modificar nada hasta comprender exactamente que me quieren decir esos NaN y no cambiare el tipo de datos de las fechas tampoco a menos de que en el futuro necesite realizar operaciones con ellos.


## Llamadas

In [ ]:
# Imprime la información general/resumida sobre el DataFrame de las llamadas

megaline_calls.info()

In [ ]:
# Imprime una muestra de datos para las llamadas

megaline_calls.sample(40)

### Corregir los datos

In [ ]:
#verificaremos si los datos estan listos para analizar
print(megaline_calls.duplicated().sum())
print(megaline_calls.isna().sum())
print(megaline_calls.nunique())

### Enriquecer los datos

In [ ]:
#Debemos cambiar el tipo de datos de la columna call_date a datetime
#Y agragar una nueva columna con los meses
megaline_calls["call_date"] = pd.to_datetime(megaline_calls["call_date"], format='%Y-%m-%d')
megaline_calls["month"] = megaline_calls["call_date"].dt.month
#verificamos como queda el dataframe
megaline_calls

In [ ]:
#Adicional a lo que hicimos anteriormente agragaremos una columna nueva que contenga las duraciones
#de las llamadas redondeadas segun la politica de la empresa

megaline_calls["rounded_duration"] = np.ceil(megaline_calls["duration"])
megaline_calls #verificamos como se ve el nuevo dataframe

## Mensajes

In [ ]:
# Imprime la información general/resumida sobre el DataFrame de los mensajes
megaline_messages.info()


In [ ]:
# Imprime una muestra de datos para los mensajes
print(megaline_messages.sample(40))

#verificaremos si los datos estan listos para analizar
print(megaline_messages.duplicated().sum())
print(megaline_messages.isna().sum())
print(megaline_messages.nunique())

### Corregir los datos

In [ ]:
#Debemos cambiar el tipo de datos de la columna message_date a datetime
megaline_messages["message_date"] = pd.to_datetime(megaline_messages["message_date"], format='%Y-%m-%d')
print(megaline_messages.head())
megaline_messages.info()

### Enriquecer los datos

In [ ]:
#Agregaremos una nueva columna de meses
megaline_messages["month"] = megaline_messages["message_date"].dt.month
#verificamos como se ve la nueva columna
megaline_messages

## Internet

In [ ]:
# Imprime la información general/resumida sobre el DataFrame de internet

megaline_internet.info()

In [ ]:
# Imprime una muestra de datos para el tráfico de internet
megaline_internet.sample(40)


### Corregir los datos

In [ ]:
#Para este dataframe realizaremos el mismo procedimiento de antes, chequear que no tenga datos ausentes,
#duplicados o extraños y ademas le cambiaremos el tipo de dato a la columna session_date a datetime
#Y adicionaremos una columna con el consumo de datos redondeado.

#verificaremos si los datos estan listos para analizar
print(megaline_internet.duplicated().sum())
print(megaline_internet.isna().sum())
print(megaline_internet.nunique())

### Enriquecer los datos

In [ ]:
#Cambiamos el tipo de datos de session_date
megaline_internet["session_date"] = pd.to_datetime(megaline_internet["session_date"], format='%Y-%m-%d')
print(megaline_internet.head())
megaline_internet.info()

In [ ]:
#creamos una nueva columna con los meses
megaline_internet["month"] = megaline_internet["session_date"].dt.month
#creamos un nuevo dataframe que contenga la informacion de los mb que utilizo cada usuario por mes
df_total_per_month = megaline_internet.groupby(["user_id", "month"]).agg(total_per_month=("mb_used", "sum"))
#Creamos una columna de que contenga el total de gb utilizado por usuario redondeando segun las normas de la empresa
df_total_per_month["gb"] = (df_total_per_month["total_per_month"] / 1024).apply(np.ceil)
df_total_per_month

## Estudiar las condiciones de las tarifas

In [ ]:
# Imprime las condiciones de la tarifa y asegúrate de que te quedan claras

megaline_plans["gb_per_month"] = np.ceil(megaline_plans["mb_per_month_included"] / 1024)
megaline_plans

Descripción de las tarifas

Nota: Megaline redondea los segundos a minutos y los megabytes a gigabytes. 

Para llamadas, cada llamada individual se redondea:

incluso si la llamada duró solo un segundo, se contará como un minuto. Para tráfico web, las sesiones web individuales no se redondean.

En vez de esto, el total del mes se redondea hacia arriba. 

Si alguien usa 1025 megabytes este mes, se le cobrarán 2 gigabytes.

A continuación puedes ver una descripción de las tarifas:

Surf
Pago mensual: 20\$.

500 minutos al mes, 50 SMS y 15 GB de datos.

Si se exceden los límites del paquete:

1 minuto: 3 centavos.

1 SMS: 3 centavos.

1 GB de datos: 10\$.

Ultimate
Pago mensual: 70\$.

3000 minutos al mes, 1000 SMS y 30 GB de datos.

Si se exceden los límites del paquete:

1 minuto: 1 centavo.

1 SMS: 1 centavo.

1 GB de datos: 7\$.

## Agregar datos por usuario



In [ ]:
# Calcula el número de llamadas hechas por cada usuario al mes. Guarda el resultado.
calls_per_month = megaline_calls.groupby(["user_id", "month"])["id"].nunique()
calls_per_month

In [ ]:
# Calcula la cantidad de minutos usados por cada usuario al mes. Guarda el resultado.
total_min_per_month = megaline_calls.groupby(["user_id", "month"]).sum()
total_min_per_month

In [ ]:
# Calcula el número de mensajes enviados por cada usuario al mes. Guarda el resultado.

total_messages_sent = megaline_messages.groupby(["user_id", "month"])["id"].nunique()
#monstramos el resultado
total_messages_sent

In [ ]:
# Calcula el volumen del tráfico de Internet usado por cada usuario al mes. Guarda el resultado.
df_total_per_month


In [ ]:
# Fusiona los datos de llamadas, minutos, mensajes e Internet con base en user_id y month

total_service = pd.concat([calls_per_month, total_min_per_month, df_total_per_month, total_messages_sent], axis=1)
total_service.columns = ["total_calls", "total_min", "rounded_min", "total_mb", "gb", "total_sms"]
total_service = total_service.reset_index()
total_service

In [ ]:
# Añade la información de la tarifa

total_service = total_service.merge(megaline_users[["plan", "user_id"]], how="left", on="user_id")
total_service = total_service.merge(megaline_plans, left_on="plan", right_on="plan_name", how="left")
total_service = total_service.drop("plan_name", axis=1)
total_service

In [ ]:
# Calcula el ingreso mensual para cada usuario

#asigno el dataframe a otra variable para conservar el original
service_calls = total_service
#sustituyo el nombre del plan por la cantidad de minutos que otorga
service_calls["plan"] = np.where((service_calls.plan == "surf"), 500, service_calls.plan)
service_calls["plan"] = np.where((service_calls.plan == "ultimate"), 3000, service_calls.plan)
#Agrego una columna nueva con la resta de los minutos 
service_calls["pay_for_calls"] = service_calls["rounded_min"] - service_calls["plan"]
#Renombro la columna plan para que quede con la informacion de minutos por usuarios
service_calls = service_calls.rename(columns={"plan":"free_min"})
#Agrego al dataframe nuevamente la columna plan 
service_calls = service_calls.merge(megaline_users[["plan", "user_id"]], how="left", on="user_id")
#Cambio el nombre de los planes por la cantidad de mensajes que otorgan
service_calls["plan"] = np.where((service_calls.plan == "surf"), 50, service_calls.plan)
service_calls["plan"] = np.where((service_calls.plan == "ultimate"), 1000, service_calls.plan)
#Renombro la columna de plan para que quede la informacion de sms por usuarios
service_calls = service_calls.rename(columns={"plan":"free_sms"})
#Agrego una columna con la resta de los sms
service_calls["pay_for_sms"] = service_calls["total_sms"] - service_calls["free_sms"]
#Agrego al dataframe nuevamente la columna plan
service_calls = service_calls.merge(megaline_users[["plan", "user_id"]], how="left", on="user_id")
#Cambio el nombre de los planes por la cantidad de gb que otorgan
service_calls["plan"] = np.where((service_calls.plan == "surf"), 15, service_calls.plan)
service_calls["plan"] = np.where((service_calls.plan == "ultimate"), 30, service_calls.plan)
#Renombro la columna de plan para que quede la informacion de gb por usuarios
service_calls = service_calls.rename(columns={"plan":"free_gb"})
#Agrego una columna con la resta de los sms
service_calls["pay_for_gb"] = service_calls["gb"] - service_calls["free_gb"]
#Agrego al dataframe nuevamente la columna plan
service_calls = service_calls.merge(megaline_users[["plan", "user_id"]], how="left", on="user_id")
#Cambio el nombre de los planes por numeros para poder realizar operaciones matematicas
service_calls["plan"] = np.where((service_calls.plan == "surf"), 0, service_calls.plan)
service_calls["plan"] = np.where((service_calls.plan == "ultimate"), 1, service_calls.plan)
#Elimine los Nan
service_calls.fillna(0, inplace=True)
service_calls

In [ ]:
service_calls["pay_for_calls_usd"] = service_calls["pay_for_calls"] * service_calls["usd_per_minute"]
service_calls["pay_for_calls_usd"] = service_calls["pay_for_calls_usd"].where(service_calls["pay_for_calls_usd"] >= 0, 0)

service_calls["pay_for_sms_usd"] = service_calls["pay_for_sms"] * service_calls["usd_per_message"]
service_calls["pay_for_sms_usd"] = service_calls["pay_for_sms_usd"].where(service_calls["pay_for_sms_usd"] >= 0, 0)

service_calls["pay_for_gb_usd"] = service_calls["pay_for_gb"] * service_calls["usd_per_gb"]
service_calls["pay_for_gb_usd"] = service_calls["pay_for_gb_usd"].where(service_calls["pay_for_gb_usd"] >= 0, 0)

service_calls["total_pay_for_user"] = service_calls["pay_for_calls_usd"] + service_calls["pay_for_sms_usd"] + service_calls["pay_for_gb_usd"] + service_calls["usd_monthly_pay"]

income_per_month = service_calls[["user_id", "month", "total_pay_for_user"]]
income_per_month

## Estudia el comportamiento de usuario

### Llamadas

In [ ]:
# Compara la duración promedio de llamadas por cada plan y por cada mes. Traza un gráfico de barras para visualizarla.
calls_mean = service_calls.pivot_table(index="month", columns="plan", values="total_min", aggfunc="mean")
calls_mean_plot = calls_mean.plot(kind="bar", title="Promedio de llamadas por plan por mes", xlabel="Meses", grid="on", figsize=[12,5])
calls_mean_plot.legend(["Surf", "Ultimate"])

In [ ]:
# Compara el número de minutos mensuales que necesitan los usuarios de cada plan. Traza un histograma.

min_need_surf = service_calls.query("plan == 0").rounded_min
min_need_ultimate = service_calls.query("plan == 1").rounded_min
min_need_surf.plot(kind="hist") #plan surf color azul en el histograma
min_need_ultimate.plot(kind="hist", title="Número de minutos que necesitan los usuarios segun su plan", figsize=[10, 5])

In [ ]:
# Calcula la media y la varianza de la duración mensual de llamadas.

min_need_surf_mean = service_calls.query("plan == 0").total_min.mean()
min_need_ultimate_mean = service_calls.query("plan == 1").total_min.mean()
min_need_surf_var = service_calls.query("plan == 0").total_min.var()
min_need_ultimate_var = service_calls.query("plan == 1").total_min.var()
print(min_need_ultimate_var)
min_need_surf_var


In [ ]:
# Traza un diagrama de caja para visualizar la distribución de la duración mensual de llamadas

box_plot = sns.boxplot(x = service_calls['month'],
            y = service_calls['total_min'])




Creo que los usuarios del plan surf aunque realizan menos llamadas que los usuarios del plan ultimate, necesitan mas minutos con respecto al plan que tienen. la variacion entre las cantidad de minutos en llamadas es pequeña en relacion de la diferencia de minutos entre un plan y el otro.

### Mensajes

In [ ]:
# Comprara el número de mensajes que tienden a enviar cada mes los usuarios de cada plan

box_plot_sms = sns.boxplot(x = service_calls['month'],
            y = service_calls['total_sms'])

In [ ]:
# Compara la cantidad de tráfico de Internet consumido por usuarios por plan

internet_trafic = service_calls.pivot_table(index="month", columns="plan", values="gb", aggfunc="sum")
internet_trafic_plot = internet_trafic.plot(kind="hist", alpha=0.6, title="Tráfico de Internet consumido por usuarios por plan", xlabel="meses", grid="on", stacked=True, figsize=[12,8])
internet_trafic_plot.legend(["Surf=0", "Ultimate=1"])



Todos los usuarios ultimate tienden a enviar mas mensajes que los de usuarios surf, sin embargo el promedio de mensajes por mes es muy bajo con respecto a la capacidad que otorga su plan, quiere decir esto que los mensajes no son el producto mas utilizado por los usuarios.

### Internet

In [ ]:
# Compara la duración promedio de uso de gb por cada plan y por cada mes. Traza un gráfico de barras para visualizarla.
internet_mean = service_calls.pivot_table(index="month", columns="plan", values="gb", aggfunc="mean")
internet_mean_plot = internet_mean.plot(kind="bar", title="Promedio de GB por plan por mes", xlabel="Meses", grid="on", figsize=[12,5])
internet_mean_plot.legend(["Surf", "Ultimate"])

In [ ]:
# Compara el número de gb mensuales que necesitan los usuarios de cada plan. Traza un histograma.

gb_need_surf = service_calls.query("plan == 0").gb
gb_need_ultimate = service_calls.query("plan == 1").gb
gb_need_surf.plot(kind="hist") #plan surf color azul en el histograma
gb_need_ultimate.plot(kind="hist", title="Número de gb que necesitan los usuarios segun su plan", figsize=[10, 5])

In [ ]:
# Calcula la media y la varianza de la duración mensual de internet.

gb_need_surf_mean = service_calls.query("plan == 0").gb.mean()
gb_need_ultimate_mean = service_calls.query("plan == 1").gb.mean()
gb_need_surf_var = service_calls.query("plan == 0").gb.var()
gb_need_ultimate_var = service_calls.query("plan == 1").gb.var()
print(gb_need_ultimate_var)
gb_need_surf_var




Se observa que los usuarios del plan ultimate consumen mas datos que los del plan surf, sin embargo a pesar de que la diferencia de gb ofrecida por cada plan es del doble, podemos pareciar que la diferencia no es tan grande con respecto al cosumo, quiere decir esto que los usuarios del plan surf consumen gb fuera del limite de su plan con mucha mas regularidad que los del plan ultimate que parecen ajustarse mas a su limite de gb.

## Ingreso

In [ ]:
income_per_month_plan = income_per_month.merge(service_calls[["plan", "user_id"]], how="left", on="user_id")
income_per_month_plan = income_per_month_plan.groupby(["plan", "month"]).agg(total_income=("total_pay_for_user", "sum"))
income_per_month_plan = income_per_month_plan.reset_index()


#income_plan_plot = income_per_month_plan.pivot_table(index="month", columns="plan", values="total_income", aggfunc="sum")
#income_plan_plot = income_plan_plot.plot(kind="hist", alpha=0.6, title="Total de ingresos por plan por mes", xlabel="Meses", figsize=[12,5])
#income_plan_plot.legend(["Surf", "Ultimate"])

#surf_income = income_per_month_plan[income_per_month_plan["plan"] == 0].groupby("month")["total_income"].sum()
#ultime_income = income_per_month_plan[income_per_month_plan["plan"] == 1].groupby("month")["total_income"].sum()
#income_hist = pd.concat([surf_income, ultime_income], axis=1)
#income_hist.plot(kind="hist", bins=6, alpha=0.8)
#plt.legend(["Surf", "Ultimate"])

#surf_income = service_calls.query("plan == 0")
#ultimate_income = service_calls.query("plan == 1")
#income_hist = pd.concat([surf_income, ultimate_income])
sns.histplot(data=service_calls, x="total_pay_for_user", hue="plan", bins=10, alpha=0.5)




Se observa que los ingresos por el plan surf son mas altos que los del plan ultimate, aunque el plan surf es mas costoso que el otro plan creemos que esto se debe a que los beneficios del plan surf se agotan mas rapido haciendo que los usuarios deban pagar tarifas adicionales para seguir disfrutando del servicio

## Prueba las hipótesis estadísticas

In [ ]:
# Prueba las hipótesis

# hipótesis alternativa "son significativamente diferentes los ingresos promedio procedentes de los usuarios de los planes de llamada Ultimate y Surf."

#Hipotesis nula " los ingresos promedio procedentes de los usuarios de los planes de llamada Ultimate y Surf. No son significativamente diferentes"

#Defino el promedio de ingresos de los usuarios por cada plan
income_per_month_plan = income_per_month.merge(service_calls[["user_id", "plan"]], how="left", on="user_id")
income_mean_surf = income_per_month_plan.query("plan == 0")
income_mean_surf = income_mean_surf.groupby("month")["total_pay_for_user"].mean()
income_mean_ultimate = income_per_month_plan.query("plan == 1")
income_mean_ultimate = income_mean_ultimate.groupby("month")["total_pay_for_user"].mean()
#Defino alpha
alpha = 0.05
#Aplico Ttest
results = st.ttest_ind(income_mean_surf, income_mean_ultimate, equal_var=False)
print("valor p:", results.pvalue)
if results.pvalue < alpha:
    print("Rechazamos la hipotesis nula")
else:
    print("No podemos rechazar la hipotesis nula")




In [ ]:
# Prueba las hipótesis
#hacemos el filtro por ciudades

income_for_city = income_per_month.merge(megaline_users[["user_id", "city"]], how="left", on="user_id")
income_for_NY = income_for_city.query("city == 'New York-Newark-Jersey City, Ny-Nj-Pa Msa'")
income_other_city = income_for_city.query("city != 'New York-Newark-Jersey City, Ny-Nj-Pa Msa'")
income_for_NY_mean = income_for_NY.groupby("month")["total_pay_for_user"].mean()
income_for_other_city_mean = income_other_city.groupby("month")["total_pay_for_user"].mean()

In [ ]:
#Probamos la hipotesis
#Hipotesis alternativa = "el ingreso promedio de los usuarios del área NY-NJ es diferente al de los usuarios de otras regiones"
#Hipotesis nula = "el ingreso promedio de los usuarios del área NY-NJ NO es diferente al de los usuarios de otras regiones"

#definimos alpha
alpha = 0.05
#Aplicamos ttest
results = st.ttest_ind(income_for_NY_mean, income_for_other_city_mean, equal_var=False)
print("valor p:", results.pvalue)
if results.pvalue < alpha:
    print("Rechazamos la hipotesis nula")
else:
    print("No podemos rechazar la hipotesis nula")

## Conclusión general



Al finalizar el analisis podemos decir que aunque los usuarios del plan surf pagan un basico mensual mas bajo, reprensentan un gran ingreso para la empresa.

Con respecto a las hipotesis rechazamos la primera hipotesis nula, ya que segun nuestros calculos la media de los ingresos por usuarios por plan son diferentes.

y la segunda hipotesis no podemos rechazarla ya que no tenemos suficiente evidencia para afirmar que las medias no son iguales entre la ciudad de NY con respecto a otras regiones.
